# Lesson 2: RAG Triad of metrics

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
import utils

import os
import openai
openai.api_key = utils.get_openai_api_key()

In [4]:
from trulens_eval import Tru

tru = Tru()
tru.reset_database()

Updating app_name and app_version in apps table: 0it [00:00, ?it/s]
Updating app_id in records table: 0it [00:00, ?it/s]
Updating app_json in apps table: 0it [00:00, ?it/s]


In [5]:
#from llama_index import SimpleDirectoryReader
from llama_index.core import SimpleDirectoryReader


documents = SimpleDirectoryReader(
    input_files=["./eBook-How-to-Build-a-Career-in-AI.pdf"]
).load_data()

In [6]:
#from llama_index import Document
from llama_index.core import Document


document = Document(text="\n\n".\
                    join([doc.text for doc in documents]))

## Replace build_sentence_window_index() & build_automerging_index()

In [7]:
# add at top of file (imports)
from llama_index.core import VectorStoreIndex, StorageContext, load_index_from_storage
from llama_index.core.node_parser import SentenceWindowNodeParser
from llama_index.core.postprocessor import MetadataReplacementPostProcessor, SentenceTransformerRerank
#from llama_index.embeddings.huggingface_openai import HuggingFaceEmbedding
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
from llama_index.embeddings.openai import OpenAIEmbedding
import os


from llama_index.core.node_parser import HierarchicalNodeParser, get_leaf_nodes

def _ensure_embed_model(embed_model):
    # allow passing either a ready embedding object or a "local:MODEL_NAME" string (your current style)
    if hasattr(embed_model, "get_text_embedding"):
        return embed_model
    if isinstance(embed_model, str):
        if embed_model.startswith("local:"):
            return HuggingFaceEmbedding(model_name=embed_model.split("local:", 1)[1])
        else:
            # treat as HF model name
            return HuggingFaceEmbedding(model_name=embed_model)
    raise ValueError("embed_model must be an embedding object or a model name string.")

def build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index",
):
    # 1) node parsing
    node_parser = SentenceWindowNodeParser.from_defaults(
        window_size=3,
        window_metadata_key="window",
        original_text_metadata_key="original_text",
    )
    nodes = node_parser.get_nodes_from_documents([document])

    # 2) ensure embedding object
    embed = _ensure_embed_model(embed_model)

    # 3) build or load index (pass llm/embed directly; no ServiceContext)
    if not os.path.exists(save_dir):
        index = VectorStoreIndex(nodes, llm=llm, embed_model=embed)
        index.storage_context.persist(persist_dir=save_dir)
    else:
        storage = StorageContext.from_defaults(persist_dir=save_dir)
        index = load_index_from_storage(storage, llm=llm, embed_model=embed)

    return index


def build_automerging_index(
    documents,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="merging_index",
    chunk_sizes=None,
):
    chunk_sizes = chunk_sizes or [2048, 512, 128]

    # 1) parse hierarchy → leaf nodes
    node_parser = HierarchicalNodeParser.from_defaults(chunk_sizes=chunk_sizes)
    nodes = node_parser.get_nodes_from_documents(documents)
    leaf_nodes = get_leaf_nodes(nodes)

    # 2) ensure embedding object
    embed = _ensure_embed_model(embed_model)

    # 3) build or load index (no ServiceContext)
    storage_context = StorageContext.from_defaults()
    storage_context.docstore.add_documents(nodes)

    if not os.path.exists(save_dir):
        index = VectorStoreIndex(
            leaf_nodes,
            storage_context=storage_context,
            llm=llm,
            embed_model=embed,
        )
        index.storage_context.persist(persist_dir=save_dir)
    else:
        storage = StorageContext.from_defaults(persist_dir=save_dir)
        index = load_index_from_storage(storage, llm=llm, embed_model=embed)

    return index


In [8]:
#from utils import build_sentence_window_index

#from llama_index.llms import OpenAI
from llama_index.llms.openai import OpenAI


llm = OpenAI(model="gpt-3.5-turbo", temperature=0.1)

sentence_index = build_sentence_window_index(
    document,
    llm,
    embed_model="local:BAAI/bge-small-en-v1.5",
    save_dir="sentence_index"
)

Loading llama_index.core.storage.kvstore.simple_kvstore from sentence_index/docstore.json.
Loading llama_index.core.storage.kvstore.simple_kvstore from sentence_index/index_store.json.


In [9]:
from utils import get_sentence_window_query_engine

sentence_window_engine = \
get_sentence_window_query_engine(sentence_index)

In [10]:
output = sentence_window_engine.query(
    "How do you create your AI portfolio?")
output.response

'You create your AI portfolio by building a collection of projects that demonstrate skill progression and complement your career goals. This portfolio should showcase a variety of successful AI projects that highlight your abilities and expertise in the field.'

## Feedback functions

In [11]:
import nest_asyncio

nest_asyncio.apply()

In [12]:
from trulens_eval import OpenAI as fOpenAI

provider = fOpenAI()

### 1. Answer Relevance

In [13]:
from trulens_eval import Feedback

f_qa_relevance = Feedback(
    provider.relevance_with_cot_reasons,
    name="Answer Relevance"
).on_input_output()

✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .


### 2. Context Relevance

In [14]:
from trulens_eval import TruLlama

context_selection = TruLlama.select_source_nodes().node.text

In [15]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input args will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input kwargs will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [16]:
import numpy as np

f_qs_relevance = (
    Feedback(provider.qs_relevance_with_cot_reasons,
             name="Context Relevance")
    .on_input()
    .on(context_selection)
    .aggregate(np.mean)
)

✅ In Context Relevance, input args will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input kwargs will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


### 3. Groundedness

In [19]:
#from trulens_eval.feedback import Groundedness

import numpy as np

# NEW imports
from trulens.core import Feedback
from trulens.providers.openai import OpenAI
from trulens.apps.llamaindex import TruLlama

# Provider (reads OPENAI_API_KEY)
provider = OpenAI()

# Groundedness feedback: context (retrieved nodes) + model output
f_groundedness = (
    Feedback(provider.groundedness_measure_with_cot_reasons, name="Groundedness")
      .on(TruLlama.select_source_nodes().node.text)  # context chunks
      .on_output()                                    # model answer
      .aggregate(np.mean)                              # aggregate statement scores
)

qa_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Answer Relevance")
      .on_input_output()
)

qs_relevance = (
    Feedback(provider.relevance_with_cot_reasons, name="Context Relevance")
      .on_input()
      .on(TruLlama.select_source_nodes().node.text)
      .aggregate(np.mean)
)

feedbacks = [qa_relevance, qs_relevance, f_groundedness]

def get_trulens_recorder(query_engine, feedbacks, app_id):
    return TruLlama(query_engine, app_id=app_id, feedbacks=feedbacks)

#grounded = Groundedness(groundedness_provider=provider)

✅ In Groundedness, input source will be set to __record__.calls[-1].rets.source_nodes[:].node.text .
✅ In Groundedness, input statement will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Answer Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Answer Relevance, input response will be set to __record__.main_output or `Select.RecordOutput` .
✅ In Context Relevance, input prompt will be set to __record__.main_input or `Select.RecordInput` .
✅ In Context Relevance, input response will be set to __record__.calls[-1].rets.source_nodes[:].node.text .


In [20]:
"""
f_groundedness = (
    Feedback(grounded.groundedness_measure_with_cot_reasons,
             name="Groundedness"
            )
    .on(context_selection)
    .on_output()
    .aggregate(grounded.grounded_statements_aggregator)
)
"""

'\nf_groundedness = (\n    Feedback(grounded.groundedness_measure_with_cot_reasons,\n             name="Groundedness"\n            )\n    .on(context_selection)\n    .on_output()\n    .aggregate(grounded.grounded_statements_aggregator)\n)\n'

## Evaluation of the RAG application

In [30]:
from trulens_eval import TruLlama
from trulens_eval import FeedbackMode

tru_recorder = TruLlama(
    sentence_window_engine,
    app_id="App_1",
    feedbacks=[
        #f_qa_relevance,
        #f_qs_relevance,
        qa_relevance,
        qs_relevance,        
        f_groundedness
    ]
)

instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.embeddings.multi_modal_base.MultiModalEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.base.embeddings.base.BaseEmbedding'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.TransformComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'llama_index.core.schema.BaseComponent'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base <class 'pydantic.main.BaseModel'>
instrumenting <class 'llama_index.embeddings.huggingface.base.HuggingFaceEmbedding'> for base

In [31]:
eval_questions = []
with open('eval_questions.txt', 'r') as file:
    for line in file:
        # Remove newline character and convert to integer
        item = line.strip()
        eval_questions.append(item)

In [32]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?']

In [33]:
eval_questions.append("How can I be successful in AI?")

In [34]:
eval_questions

['What are the keys to building a career in AI?',
 'How can teamwork contribute to success in AI?',
 'What is the importance of networking in AI?',
 'What are some good habits to develop for a successful career?',
 'How can altruism be beneficial in building a career?',
 'What is imposter syndrome and how does it relate to AI?',
 'Who are some accomplished individuals who have experienced imposter syndrome?',
 'What is the first step to becoming good at AI?',
 'What are some common challenges in AI?',
 'Is it normal to find parts of AI challenging?',
 'How can I be successful in AI?']

In [35]:
for question in eval_questions:
    with tru_recorder as recording:
        sentence_window_engine.query(question)

/Users/ytchen/Documents/experimental/AdvancedRAG/.venv/lib/python3.11/site-packages/trulens/feedback/llm_provider.py:1966: UserWarning: Failed to process and remove trivial statements. Proceeding with all statements.
  warnings.warn(


In [41]:
records, feedback = tru.get_records_and_feedback(app_ids=[])
records.head()


app_name app_version                                     app_id  \
0    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
1    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
2    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
3    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   
4    App_1        base  app_hash_9a8bbe93b76fc0a0eca1a6fc7bd3027e   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                             app_json  \
0  {'tru_class_info': {'name': 'TruLlama', 'module': {'package_name': 'trulens.apps.llamaindex', 'module_name': 'trulens.apps.llamaindex.tru_llama'}, 'bases': [{'name': 'TruLlama', 'module': {'package_name': 'trulens.apps.llamaindex', 'module_name': 'trulens.apps.llamaindex.tru_llama'}, 'bases': None}, {'name': 'App', 'module': {'package_name': 'trulens.core', 'module_name': 'trulens.core.app'}, 'bases': None}, {'name': 'AppDefinition', 'module': {'package_name': 'trulens.core.schema', 'module_name': 'trulens.core.schema.app'}, 'bases': None}, {'name': 'WithClassInfo', 'module': {'package_name': 'trulens.core.utils', 'module_name': 'trulens.core.utils.pyschema'}, 'bases': None}, {'name': 'SerialModel', 'module': {'package_name': 'trulens.core.utils', 'module_name': 'trulens.core.utils.serial'}, 'bases': None}, {'name': 'BaseModel', 'module': {'package_name': 'pydantic', 'module_name': 'pydantic.main'}, 'bases': None}, {'name': 'WithInstrumentCallbacks', 'module': {'package_name': 'trulens.core', 'module_name': 'trulens.core.instruments'}, 'bases': None}, {'name': 'Hashable', 'modul

In [38]:
import pandas as pd

pd.set_option("display.max_colwidth", None)
records[["input", "output"] + feedback]

,input,output,Groundedness,Answer Relevance,Context Relevance
0,How can I be successful in AI?,"To be successful in AI, it is important to have a strong foundation in technical skills such as programming, statistics, and machine learning. Additionally, gaining a deeper understanding of specific application areas like natural language processing or computer vision, as well as technology areas such as probabilistic graphical models and scalable software systems, can further enhance your expertise in the field. Continuous learning and skill progression through practical projects are also key components to success in AI.",0.666667,1.000000,0.833333
1,Is it normal to find parts of AI challenging?,It is normal to find parts of AI challenging.,1.000000,1.000000,0.833333
2,What are some common challenges in AI?,"Some common challenges in AI include keeping up-to-date with rapidly evolving technologies, the iterative nature of AI projects leading to uncertainties in project management, the need to collaborate with stakeholders who may lack expertise in AI, difficulties in finding suitable projects and estimating timelines, and the ongoing learning process required for mastering foundational skills.",1.000000,1.000000,0.500000
3,What is the first step to becoming good at AI?,Identifying possible roles and conducting informational interviews.,0.500000,0.000000,0.666667
4,Who are some accomplished individuals who have experienced imposter syndrome?,"Former Facebook COO Sheryl Sandberg, U.S. first lady Michelle Obama, actor Tom Hanks, and Atlassian co-CEO Mike Cannon-Brookes.",0.333333,1.000000,0.666667
5,What is imposter syndrome and how does it relate to AI?,"Imposter syndrome is when an individual doubts their accomplishments and has a persistent fear of being exposed as a fraud. In the context of AI, newcomers to the field may experience imposter syndrome, feeling like they do not truly belong in the AI community despite their success. This phenomenon can be discouraging but is a common experience among many individuals, including well-known figures like Sheryl Sandberg and Michelle Obama.",0.916667,1.000000,0.333333
6,How can altruism be beneficial in building a career?,Helping others during one's career journey can lead to better outcomes for oneself.,0.500000,0.666667,0.166667
7,What are some good habits to develop for a successful career?,"Developing good habits in areas such as eating, exercise, sleep, personal relationships, work, learning, and self-care can help individuals move forward in their careers while maintaining their health. Additionally, aiming to lift others during each step of one's own journey can lead to better outcomes.",0.666667,1.000000,0.666667
8,What is the importance of networking in AI?,"Networking in AI is crucial as it can provide valuable insights, guidance, and opportunities for individuals looking to advance in the field. By connecting with professionals who have experience in AI, individuals can gain knowledge about the industry, potential career paths, and current trends. Networking also allows for the exchange of information, ideas, and resources, which can help individuals stay informed and competitive in the rapidly evolving field of AI. Additionally, building a strong network can lead to potential job opportunities and collaborations that may not have been otherwise accessible.",0.750000,1.000000,0.166667
9,How can teamwork contribute to success in AI?,"Teamwork can contribute to success in AI by allowing individuals to leverage the diverse skills and perspectives of their colleagues. Collaborating with others can lead to more innovative solutions, effective problem-solving, and the ability to tackle larger projects that require a combination of technical expertise and leadership skills. Additionally, working in a team can create a supportive environment where team members can learn from each other, share insights, and collectively drive projects forward towards successful outcomes.",0.666667,1.0

In [39]:
tru.get_leaderboard(app_ids=[])

,,Answer Relevance,Context Relevance,Groundedness,latency,total_cost
app_name,app_version,,,,,
App_1,base,0.909091,0.5,0.690657,1.362364,0.000808


In [40]:
tru.run_dashboard()

Starting dashboard ...


/var/folders/7t/drc_5mxn3c96k2_3zc1n57yh0000gn/T/ipykernel_80493/1552380249.py:1: DeprecationWarning: Method `run_dashboard` has been renamed or moved to `trulens.dashboard.run.run_dashboard`.

  tru.run_dashboard()
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Accordion(children=(VBox(children=(VBox(children=(Label(value='STDOUT'), Output())), VBox(children=(Label(valu…

Dashboard started at http://localhost:59794 .


<Popen: returncode: None args: ['streamlit', 'run', '--server.headless=True'...>